In [1]:
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from feature_extractor import FeatureExtractor
import urllib.request
import matplotlib.pyplot as plt
from PIL import Image
import requests
import numpy as np
from io import BytesIO, StringIO
import cv2
from tqdm import tqdm
import glob
import tensorflow as tf

**Creation of an image generator from the csv to load it in a light way**

In [2]:
class My_Custom_Generator(tf.keras.utils.Sequence) :
    
  
    def __init__(self, image_filenames, batch_size) :
        self.image_filenames = image_filenames
        self.batch_size = batch_size
        self.empty_images = []
      
    def util_func(self, img_url):
        try:
              return Image.open(BytesIO(requests.get(img_url).content))
        except:
              self.empty_images.append(img_url)
    
    
    def __len__(self) :
        return int((np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int))
  
  
    def __getitem__(self, idx) :
            batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
            
            return np.array([ (self.util_func(file_name), file_name) for file_name in batch_x if self.util_func(file_name)!=None])

**Initiation of the feature extractor**

In [3]:
fe = FeatureExtractor()

**feature extractor with storing the dataframe each batch (sometimes, due to an unknown error, the system is blocked and all data is lost)**

In [ ]:
for count, csv_path in enumerate(glob.glob('./data/ready_to_extract_sub_dataset_*.csv')):
    print(count+1)
    df = pd.read_csv(csv_path)    
    df[["f {}".format(i+1) for i in range(4096) ]] = np.nan
    data = My_Custom_Generator(df['OriginalURL'].values, 94)    
    for batch in tqdm(data, desc ="Executed task"):
        features = fe.extract_batch([sub[0] for sub in batch])
        df.loc[[df.index[df['OriginalURL'] == x][0] for x in [sub[1] for sub in batch]],["f {}".format(i+1) for i in range(4096) ]] = features
        df = df[~df['OriginalURL'].isin(data.empty_images)]
    df.to_csv('./data/dataset_with_features/extracted_features_sub_dataset_{}.csv'.format(count+1))

1


c:\python38\lib\site-packages\pandas\core\frame.py:3645: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = value
C:\Users\AIM2021\AppData\Local\Temp/ipykernel_3412/3607583605.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return int((np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np